In [2]:
!pip install -q transformers datasets tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
train_dataset = dataset['train']
test_dataset = dataset['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 10.1 MB/s eta 0:00:00
Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [3]:
def get_model_prediction(question, options_dict):
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"Question: {question}\nOptions: {options_text}\nThink like a medical professional step by step.\nAnswer:"

    # print(input_text)
    # print("***********")

    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (can be tuned using temperature, max_length, etc.)
    output = model.generate(inputs["input_ids"], max_new_tokens=500)


    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # print(generated_text)
    # print("***********")

    ind=generated_text.find('Options')
    extracted_answer=generated_text[ind+len(options_text):]
    # print(extracted_answer)
    return extracted_answer
    # match = re.search(r"Answer:\s*([A-D])", generated_text)

    # if match:
    #     predicted_option = match.group(1)
    # else:
    #     predicted_option = None

    # return predicted_option

In [ ]:
# Evaluate model on the test dataset with tqdm to monitor the progress
correct_predictions = 0
total_predictions = 0

# Add tqdm to track the evaluation loop
for sample in tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset)):
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options)
    # print("------------------------")
    # print(correct_answer)
    # Check if prediction is correct
    if correct_answer.lower() in predicted_answer.lower():
        correct_predictions += 1
    # print(predicted_answer)
    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {total_predictions}: Accuracy so far: {accuracy:.2f}%")


# Final accuracy
final_accuracy = correct_predictions / total_predictions * 100
print(f"Final Accuracy: {final_accuracy:.2f}%")

Evaluating on test dataset:   0%|          | 1/1273 [01:17<27:32:46, 77.96s/it]

Iteration 1: Accuracy so far: 100.00%


Evaluating on test dataset:   0%|          | 2/1273 [02:10<22:16:03, 63.07s/it]

Iteration 2: Accuracy so far: 50.00%


Evaluating on test dataset:   0%|          | 3/1273 [03:55<28:55:49, 82.01s/it]

...

Iteration 109: Accuracy so far: 86.24%


Evaluating on test dataset:   9%|▊         | 110/1273 [2:59:51<30:46:06, 95.24s/it]

Iteration 110: Accuracy so far: 86.36%


Evaluating on test dataset:   9%|▊         | 111/1273 [3:01:15<29:41:07, 91.97s/it]

Iteration 111: Accuracy so far: 86.49%


In [4]:

total_predictions = 111
accuracy_at_111 = 86.49
correct_predictions =   int((accuracy_at_111/100) * total_predictions)

# Add tqdm to track the evaluation loop
for i,sample in enumerate(tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset))):
    if i<111:
      continue

    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options)
    # print("------------------------")
    # print(correct_answer)
    # Check if prediction is correct
    if correct_answer.lower() in predicted_answer.lower():
        correct_predictions += 1
    # print(predicted_answer)
    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {total_predictions}: Accuracy so far: {accuracy:.2f}%")


# Final accuracy
final_accuracy = correct_predictions / total_predictions * 100
print(f"Final Accuracy: {final_accuracy:.2f}%")

Evaluating on test dataset:   0%|          | 0/1273 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.
Evaluating on test dataset:   9%|▉         | 112/1273 [01:14<12:51,  1.51it/s]

Iteration 112: Accuracy so far: 86.61%


Evaluating on test dataset:   9%|▉         | 113/1273 [02:30<31:06,  1.61s/it]

Iteration 113: Accuracy so far: 86.73%


Evaluating on test dataset:   9%|▉         | 114/1273 [03:23<48:49,  2.53s/it]

...

Iteration 152: Accuracy so far: 86.84%


Evaluating on test dataset:  12%|█▏        | 153/1273 [1:04:11<32:39:26, 104.97s/it]

Iteration 153: Accuracy so far: 86.93%


Evaluating on test dataset:  12%|█▏        | 153/1273 [1:04:29<7:52:05, 25.29s/it]  


KeyboardInterrupt: 